In [ ]:
# %% Silenciar logs de todo (incl. Earth Engine) y mostrar solo la barra
import os, logging, warnings
import multiprocessing as mp

import geopandas as gpd
import ee
import rasterio as rio
from tqdm.auto import tqdm

from utils.cloud_detection import (
    get_s1_col,
    get_s2cloudless_collection,
    get_matching_s1_s2,
    get_images,
)

# ---- parámetros de descarga
clean_threshold = 0.9
patch_size      = 64
S2_BANDS        = ["B4","B3","B2","B8","B11","B12"]
S1_BANDS        = ["VV","VH"]
start_date      = "2021-01-01"
end_date        = "2021-12-31"

# ---- util MP
def _get_mp_ctx():
    try:
        return mp.get_context("fork") if "fork" in mp.get_all_start_methods() else mp.get_context("spawn")
    except Exception:
        return mp

# ---- inicializador por worker (sin prints)
def _init_worker(quiet):
    if quiet:
        _silence_everything()
    ee.Initialize(opt_url=None)

# ---- función para silenciar librerías ruidosas
def _silence_everything():
    warnings.filterwarnings("ignore")
    logging.captureWarnings(True)

    # Nivel global
    logging.basicConfig(level=logging.CRITICAL)
    logging.getLogger().setLevel(logging.CRITICAL)

    # Loggers específicos que suelen hablar en EE / Google
    for name in [
        "ee", "earthengine", "earthengine-api",
        "google", "googleapiclient", "googleapiclient.discovery",
        "googleapiclient.http", "google.auth", "oauth2client",
        "urllib3", "fiona", "rasterio"
    ]:
        try:
            logging.getLogger(name).setLevel(logging.CRITICAL)
            logging.getLogger(name).propagate = False
        except Exception:
            pass

# ---- worker
def process_centroid(task):
    """
    task = (idx, lat, lon, out_s1_dir, out_s2_dir)
    return: (True/False, idx)  (solo lo necesario para la barra)
    """
    idx, lat, lon, out_s1_dir, out_s2_dir = task
    name = str(idx).zfill(5)
    try:
        col_s2 = get_s2cloudless_collection(lat, lon, start_date, end_date,
                                            clean_threshold, patch_size)
        col_s1 = get_s1_col(lat, lon, start_date, end_date,
                            patch_size, orbit_pass='DESCENDING')
        m_s1, m_s2, _ = get_matching_s1_s2(col_s1, col_s2)
        if not m_s1 or not m_s2:
            return (False, idx)

        m_s1 = m_s1[0]; m_s2 = m_s2[0]
        s2_imgs, s2_prof = get_images(lat, lon, [m_s2], S2_BANDS, patch_size)
        s1_imgs, s1_prof = get_images(lat, lon, [m_s1], S1_BANDS, patch_size)
        s2_arr = s2_imgs[0]["image"]; s1_arr = s1_imgs[0]["image"]

        os.makedirs(out_s1_dir, exist_ok=True)
        os.makedirs(out_s2_dir, exist_ok=True)
        s1_name = f"s1_{name}.tif"; s2_name = f"s2_{name}.tif"
        with rio.open(os.path.join(out_s1_dir, s1_name), "w", **s1_prof) as dst:
            dst.write(s1_arr)
        with rio.open(os.path.join(out_s2_dir, s2_name), "w", **s2_prof) as dst:
            dst.write(s2_arr)

        return (True, idx)
    except Exception:
        return (False, idx)

# ---- lanzador con SOLO la barra
def run_downloads(centroids_gj_path, out_s1_dir, out_s2_dir,
                  max_workers=24, chunksize=16, quiet=True):
    if quiet:
        _silence_everything()

    gdf = gpd.read_file(centroids_gj_path).to_crs(epsg=4326)
    tasks = [(i, row.geometry.y, row.geometry.x, out_s1_dir, out_s2_dir)
             for i, (_, row) in enumerate(gdf.iterrows())]
    total = len(tasks)

    # inicializa EE una vez en el main
    ee.Initialize()

    n_workers = min(mp.cpu_count(), max_workers)
    ctx = _get_mp_ctx()

    ok = 0
    err = 0

    with ctx.Pool(processes=n_workers, initializer=_init_worker, initargs=(quiet,)) as pool:
        with tqdm(total=total, unit="tile", dynamic_ncols=True, desc="Descargando") as pbar:
            for success, _ in pool.imap_unordered(process_centroid, tasks, chunksize=chunksize):
                if success:
                    ok += 1
                else:
                    err += 1
                pbar.update(1)
                # postfix minimalista con conteos
                pbar.set_postfix(ok=ok, err=err)

In [ ]:
centroids_geojson = "../dataset_tfm/spain.geojson"
out_s2_dir = "../dataset_tfm/s2"
out_s1_dir = "../dataset_tfm/s1"
run_downloads(centroids_geojson, out_s1_dir, out_s2_dir, max_workers=28, chunksize=16, quiet=True)

In [ ]:
import pathlib
DB_FOLDER = pathlib.Path("/home/tidop/masterIA/TFM_BRCD/dataset_tfm")

s2_files = sorted(list(DB_FOLDER.glob("**/s2/*.tif")))
s1_files = sorted(list(DB_FOLDER.glob("**/s1/*.tif")))

len(s2_files), len(s1_files)

In [ ]:
## Now, filter the files with NAN values and black areas
s2_filtered = []
s1_filtered = []

import rasterio as rio
import numpy as np

def is_valid_image(image_path):
    with rio.open(image_path) as src:
        data = src.read()
        # Check for NaN values and if has values equal to zero across the bands
        if (data == 0).all() or np.isnan(data).any():
            return False
    return True

for i, (s2_file, s1_file) in enumerate(zip(s2_files, s1_files)):
    if is_valid_image(s2_file) and is_valid_image(s1_file):
        s2_filtered.append(s2_file)
        s1_filtered.append(s1_file)
    else:
        print(f"Invalid file pair: {s2_file}, {s1_file}")

    print(f"Processed {i+1}/{len(s2_files)}, valid pairs found: {len(s2_filtered)}", end="\r")

In [ ]:
### shutil copy 
import shutil
destination_folder = DB_FOLDER / "filtered_data"
s1_destination_folder = destination_folder / "s1"
s2_destination_folder = destination_folder / "s2"
s1_destination_folder.mkdir(parents=True, exist_ok=True)
s2_destination_folder.mkdir(parents=True, exist_ok=True)
for s2_file, s1_file in zip(s2_filtered, s1_filtered):
    shutil.copy(s2_file, s2_destination_folder)
    shutil.copy(s1_file, s1_destination_folder)


In [ ]:
from multiprocessing import Pool, cpu_count

ee.Initialize()
n_workers = min(cpu_count(), 24)
with Pool(processes=n_workers) as pool:
    for i, res in enumerate(pool.imap_unordered(process_centroid, tasks), start=1):
        if res is None:
            continue
        name, s1_fname, s2_fname = res
        print(f"[{i}/{len(tasks)}] {name} → S1: {s1_fname}, S2: {s2_fname}")

In [ ]:
import os
import time
import math
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.windows import from_bounds
from rasterio.features import geometry_mask
from shapely.geometry import shape, mapping
from shapely.validation import make_valid
from shapely.ops import unary_union
import multiprocessing as mp
from tqdm.auto import tqdm

# ===== Configuración =====
AREA_THRESHOLD = 0.05  # 5 %
ALL_TOUCHED = False     # Solo píxeles cuyo centro está dentro de la geometría
CHUNKSIZE = 256         # Cuántas features procesa cada worker de golpe

# ===== Funciones utilitarias =====
def _get_mp_ctx():
    return mp.get_context('fork') if 'fork' in mp.get_all_start_methods() else mp.get_context('spawn')

def _clean_geom(geom):
    if geom.is_empty:
        return None
    g = make_valid(geom)
    if g.is_empty:
        return None
    if g.geom_type == "GeometryCollection":
        polys = [p for p in g.geoms if p.geom_type in ("Polygon", "MultiPolygon")]
        if not polys:
            return None
        g = unary_union(polys)
    return g if g.is_valid and not g.is_empty else None

# ===== Variables globales para multiprocessing =====
_RDS = None
_PIXEL_AREA = None

def _init_worker(raster_path):
    global _RDS, _PIXEL_AREA
    _RDS = rasterio.open(raster_path)
    # Calcula área de un píxel en m²
    px_w = abs(_RDS.transform.a)
    px_h = abs(_RDS.transform.e)
    _PIXEL_AREA = px_w * px_h

def _process_feature(feat):
    global _RDS, _PIXEL_AREA
    shp = _clean_geom(feat.geometry)
    if shp is None:
        return None

    cell_area = shp.area
    if cell_area <= 0 or not math.isfinite(cell_area):
        return None

    try:
        window = from_bounds(*shp.bounds, transform=_RDS.transform)
        arr = _RDS.read(1, window=window, boundless=True, masked=True)
    except Exception:
        return None

    if arr.size == 0:
        return None

    win_transform = rasterio.windows.transform(window, _RDS.transform)
    mask_inside = geometry_mask(
        [mapping(shp)],
        out_shape=arr.shape,
        transform=win_transform,
        invert=True,
        all_touched=ALL_TOUCHED
    )

    valid_pixels = np.logical_and(mask_inside, ~arr.mask) if hasattr(arr, "mask") else mask_inside
    pix1 = np.logical_and(valid_pixels, arr == 1)
    built_area = pix1.sum() * _PIXEL_AREA

    if (built_area / cell_area) >= AREA_THRESHOLD:
        return feat
    return None

# ===== Función principal =====
def filtrar_grilla_por_raster(vector_path, raster_path, output_path, processes=None):
    start_time = time.time()

    # Leer raster
    with rasterio.open(raster_path) as rds:
        raster_crs = rds.crs

    # Leer vector
    gdf = gpd.read_file(vector_path)

    # Reproyectar si es necesario
    if gdf.crs != raster_crs:
        print(f"Reproyectando vector de {gdf.crs} a {raster_crs}")
        gdf = gdf.to_crs(raster_crs)

    if processes is None:
        processes = max(1, mp.cpu_count() - 1)

    ctx = _get_mp_ctx()
    results = []
    with ctx.Pool(processes=processes, initializer=_init_worker, initargs=(raster_path,)) as pool:
        for res in tqdm(pool.imap_unordered(_process_feature, [row for _, row in gdf.iterrows()], chunksize=CHUNKSIZE),
                        total=len(gdf), unit="feat", dynamic_ncols=True):
            if res is not None:
                results.append(res)

    # Guardar resultado
    out_gdf = gpd.GeoDataFrame(results, columns=gdf.columns, crs=raster_crs)
    out_gdf.to_file(output_path, driver="GeoJSON")

    elapsed = time.time() - start_time
    print(f"✅ Listo. Total: {len(gdf)} | Conservados: {len(out_gdf)} (≥ {AREA_THRESHOLD*100:.1f} % área)")
    print(f"⏱️ {elapsed:.1f}s | {len(gdf)/elapsed:,.0f} feat/s")
    print(f"➡️ {output_path}")

In [ ]:
vector_in = "../brcd_data/whole_spain_grided_builtup.geojson"
raster_in = "../brcd_data/Built-up_epsg3035_2.5m.tif"
salida = "../brcd_data/spain_filtrada.geojson"
filtrar_grilla_por_raster(vector_in, raster_in, salida)

In [ ]:
## Filter if has 0 or NAN both S2 and S1 file
import pathlib
s2_files = sorted(list(pathlib.Path(out_s2).glob("*.tif")))
s1_files = sorted(list(pathlib.Path(out_s1).glob("*.tif")))

for i, (s2_file, s1_file) in enumerate(zip(s2_files, s1_files)):
    with rio.open(s2_file) as src:
        s2_data = src.read()
        if (s2_data == 0).all() or (s2_data != s2_data).all():
            print(f"[FILTER] Removing {s2_file} due to all zeros or NaNs")
            s2_file.unlink()
            continue

    with rio.open(s1_file) as src:
        s1_data = src.read()
        if (s1_data == 0).all() or (s1_data != s1_data).all():
            print(f"[FILTER] Removing {s1_file} due to all zeros or NaNs")
            s1_file.unlink()
            continue

print(f"[INFO] Filtered {len(s2_files) - len(list(pathlib.Path(out_s2).glob('*.tif')))} S2 files and {len(s1_files) - len(list(pathlib.Path(out_s1).glob('*.tif')))} S1 files.")

In [ ]:
## Download building
import pathlib
import rasterio as rio

DB_FOLDER = pathlib.Path("../brcd")

building_files = sorted(list((DB_FOLDER / "building").glob("*.tif")))
road_files = sorted(list((DB_FOLDER / "road").glob("*.tif")))

## Filter if has less than 2.5% of non-zero pixels (both building and road)

building_filtered_files = []
road_filtered_files = []

for i, (building_file, road_file) in enumerate(zip(building_files, road_files)):
    with rio.open(building_file) as src1, rio.open(road_file) as src2:
        building_data = src1.read()
        road_data = src2.read()

        # Calculate non-zero pixel percentages
        building_non_zero = (building_data != 0).sum() / building_data.size
        road_non_zero = (road_data != 0).sum() / road_data.size

        if building_non_zero >= 0.025 and road_non_zero >= 0.025:
            building_filtered_files.append(building_file)
            road_filtered_files.append(road_file)

            print(f"[{i+1}/{len(building_files)}] Keeping {building_file} and {road_file} with {building_non_zero:.2%} building and {road_non_zero:.2%} road non-zero pixels.")


In [ ]:
len(building_filtered_files), len(road_filtered_files)

In [ ]:
import shutil

## Create new folders with
NEW_FOLDER = pathlib.Path("../dataset_brcd")
NEW_FOLDER.mkdir(parents=True, exist_ok=True)

S1_FOLDER = NEW_FOLDER / "s1"
S2_FOLDER = NEW_FOLDER / "s2"
BUILDING_FOLDER = NEW_FOLDER / "building"   
ROAD_FOLDER = NEW_FOLDER / "road"

S1_FOLDER.mkdir(parents=True, exist_ok=True)
S2_FOLDER.mkdir(parents=True, exist_ok=True)
BUILDING_FOLDER.mkdir(parents=True, exist_ok=True)
ROAD_FOLDER.mkdir(parents=True, exist_ok=True)

for i, (building_file, road_file) in enumerate(zip(building_filtered_files, road_filtered_files)):  
    s1_file = building_file.parent.parent / "s1" / building_file.name.replace("building", "s1")
    s2_file = building_file.parent.parent / "s2" / building_file.name.replace("building", "s2")

    s1_dest = S1_FOLDER / s1_file.name
    s2_dest = S2_FOLDER / s2_file.name   
    building_dest = BUILDING_FOLDER / building_file.name
    road_dest = ROAD_FOLDER / road_file.name

    if s1_file.exists() and s2_file.exists():
        shutil.copy(s1_file, s1_dest)
        shutil.copy(s2_file, s2_dest)
        shutil.copy(building_file, building_dest)
        shutil.copy(road_file, road_dest)
        print(f"[{i+1}/{len(building_filtered_files)}] Copied {s1_file.name}, {s2_file.name}, {building_file.name}, and {road_file.name} to new dataset folder.")

In [ ]:
## Unlink files with NAN VALUES in s2 or s1 files
import pathlib
s2_files = sorted(list(pathlib.Path(S2_FOLDER).glob("*.tif")))
s1_files = sorted(list(pathlib.Path(S1_FOLDER).glob("*.tif")))

non_nan_s2_files = []
non_nan_s1_files = []
for i, (s2_file, s1_file) in enumerate(zip(s2_files, s1_files)):

    with rio.open(s2_file) as src1, rio.open(s1_file) as src2:
        s2_data = src1.read()
        s1_data = src2.read()

        # Check for NaN values
        if np.isnan(s2_data).any() or np.isnan(s1_data).any():
           continue
        else:
            non_nan_s2_files.append(s2_file)
            non_nan_s1_files.append(s1_file)
            print(f"[{i+1}/{len(s2_files)}] Keeping {s2_file} and {s1_file} with no NaN values.")

In [ ]:
len(non_nan_s2_files), len(non_nan_s1_files)

In [ ]:
print(f"S2 Mean: {s2_mean/10_000}, S2 Std: {s2_std/10_000}")
print(f"S1 Mean: {s1_mean}, S1 Std: {s1_std}")

In [ ]:
import rasterio as rio
import numpy as np
import pathlib

def compute_dataset_class_percentages(tif_paths, building_value=1, road_value=2):
    """
    Calcula el porcentaje promedio de píxeles de edificios y carreteras
    en un conjunto de archivos TIFF de etiquetas.

    Args:
        tif_paths (list[str]): Lista de rutas a archivos TIFF de máscara.
        building_value (int): Valor de pixel que codifica edificios en la máscara.
        road_value (int): Valor de pixel que codifica carreteras en la máscara.

    Returns:
        tuple: (avg_building, avg_road) porcentajes promedio sobre todos los píxeles.
    """
    total_pixels = 0
    total_class = 0

    for path in tif_paths:
        with rio.open(path) as src:
            mask = src.read(1)  # lee la primera banda de la máscara
        total_pixels += mask.size
        total_class += np.count_nonzero(mask == 1)

    avg_class = total_class / total_pixels
    return avg_class

# Ejemplo de llamada:
BUILDING_FOLDER = "../dataset_brcd/road"
tif_list = sorted(list(pathlib.Path(BUILDING_FOLDER).glob("*.tif")))
avg_c = compute_dataset_class_percentages(tif_list)
print(f"Promedio clase: {avg_c:.4f}")

In [2]:
from overturemaps import core
import rasterio as rio
from rasterio.features import rasterize
import pyproj
from shapely.geometry import box
from multiprocessing import Pool
from tqdm.auto import tqdm
from pathlib import Path
import warnings

warnings.filterwarnings("ignore", message="invalid value encountered in area")
warnings.filterwarnings("ignore", category=FutureWarning)

# Buffers por clase
buffer_dict = {
    'motorway': 3.5, 'motorway_link': 2.5,
    'trunk': 3.5, 'trunk_link': 2.5,
    'primary': 3.5, 'primary_link': 2.5,
    'secondary': 3.5, 'secondary_link': 2.5,
    'tertiary': 3.5, 'tertiary_link': 2.5,
    'residential': 2.5, 'living_street': 2.5,
    'unclassified': 1.25, 'pedestrian': 1.25,
    'footway': 1.25, 'cycleway': 1.25,
    'service': 2.5, 'steps': 1.25,
    'path': 1.25, 'track': 1.25, 'bridleway': 1.25,
    'busway': 2.5, 'unknown': 1.25
}

def create_masks(s2_path, out_tif_building, out_tif_road, pixel_size=2.5, out_shape=(256, 256)):
    """
    Crea las máscaras (building/road). Devuelve dict: {'status': 'ok'|'skipped'|'error', 'msg': str}
    """
    s2_path = Path(s2_path); out_tif_building = Path(out_tif_building); out_tif_road = Path(out_tif_road)
    try:
        # Omitir si ambos ya existen
        if out_tif_building.exists() and out_tif_road.exists():
            return {"status": "skipped", "msg": f"{out_tif_building.name} y {out_tif_road.name} existen, se omite."}

        with rio.open(s2_path) as src:
            minx, miny, maxx, maxy = src.bounds
            transformer = pyproj.Transformer.from_crs(src.crs, "EPSG:4326", always_xy=True)
            minx_wgs, miny_wgs = transformer.transform(minx, miny)
            maxx_wgs, maxy_wgs = transformer.transform(maxx, maxy)
            bbox_wgs84 = (minx_wgs, miny_wgs, maxx_wgs, maxy_wgs)

            # Overture
            building_gdf = core.geodataframe("building", bbox=bbox_wgs84); building_gdf.crs = "EPSG:4326"
            road_gdf     = core.geodataframe("segment",  bbox=bbox_wgs84); road_gdf.crs     = "EPSG:4326"

            # Reproyección + recorte
            bbox_geom = box(minx, miny, maxx, maxy)
            building_gdf = building_gdf.to_crs(src.crs)
            building_gdf = building_gdf[building_gdf.geometry.intersects(bbox_geom)]
            building_gdf = building_gdf[building_gdf.geometry.area > 100]  # filtra edificios pequeños

            road_gdf = road_gdf.to_crs(src.crs)
            road_gdf = road_gdf[road_gdf.geometry.intersects(bbox_geom)]
            if 'class' in road_gdf.columns:
                road_gdf['buffer'] = road_gdf['class'].map(buffer_dict).fillna(1.25)
            else:
                road_gdf['buffer'] = 1.25
            road_gdf['geometry'] = road_gdf.apply(lambda r: r['geometry'].buffer(r['buffer']), axis=1)

            # Transform y rasterización
            transform = rio.Affine(pixel_size, 0, minx, 0, -pixel_size, maxy)
            building_raster = rasterize([(g, 1) for g in building_gdf.geometry], out_shape=out_shape, transform=transform, fill=0, dtype='uint8')
            road_raster     = rasterize([(g, 1) for g in road_gdf.geometry],     out_shape=out_shape, transform=transform, fill=0, dtype='uint8')

            meta = dict(driver='GTiff', height=out_shape[0], width=out_shape[1], count=1, dtype='uint8', crs=src.crs, transform=transform)
            out_tif_building.parent.mkdir(parents=True, exist_ok=True)
            out_tif_road.parent.mkdir(parents=True, exist_ok=True)
            with rio.open(out_tif_building, 'w', **meta) as dst: dst.write(building_raster, 1)
            with rio.open(out_tif_road, 'w', **meta) as dst:     dst.write(road_raster, 1)

        return {"status": "ok", "msg": f"Procesado {s2_path.name}"}

    except Exception as e:
        return {"status": "error", "msg": f"Error en {s2_path}: {e}"}

def _worker(args):
    return create_masks(*args)

def split_existing(pairs, show_progress=True):
    """
    Separa pares en existentes (ambos outputs ya están) y a_procesar.
    Devuelve existing_pairs, todo_pairs y muestra una barra de verificación.
    """
    existing, todo = [], []
    if show_progress:
        pbar = tqdm(pairs, desc="Verificando existentes", unit="par")
    else:
        pbar = pairs
    count_exist = 0
    for item in pbar:
        s2, out_b, out_r = item
        if Path(out_b).exists() and Path(out_r).exists():
            existing.append(item); count_exist += 1
        else:
            todo.append(item)
        if show_progress:
            tqdm.write("", end="")  # mantener limpia la barra en algunos entornos
            pbar.set_postfix(existentes=count_exist, por_hacer=len(todo), refresh=True)
    if show_progress and hasattr(pbar, "close"):
        pbar.close()
    return existing, todo

def parallel_create_masks(
    pairs,
    processes=16,
    pixel_size=2.5,
    out_shape=(256, 256),
    show_progress=True,
    desc="Procesando (descarga + rasterización)"
):
    """
    Ejecuta con:
      - Barra 1: conteo de existentes (skip)
      - Barra 2: procesamiento de faltantes con ETA, errores y descargados
    """
    # Barra 1: contar existentes
    existing_pairs, todo_pairs = split_existing(pairs, show_progress=show_progress)

    # Crear resultados "skipped" para existentes (para resumen final)
    skipped_results = [
        {"status": "skipped", "msg": f"{Path(b).name} y {Path(r).name} ya existen"}
        for _, b, r in existing_pairs
    ]

    # Si no hay nada que hacer, mostramos un mini-resumen y salimos
    if not todo_pairs:
        if show_progress:
            tqdm.write(f"Todo listo: {len(existing_pairs)} ya existentes, 0 por procesar.")
        return skipped_results

    # Preparar argumentos con parámetros fijos
    todo_args = [(*p, pixel_size, out_shape) for p in todo_pairs]

    done = 0
    errors = 0
    results = []

    if processes is None or processes <= 1:
        # Secuencial
        pbar = tqdm(total=len(todo_args), disable=not show_progress, desc=desc, unit="par")
        for args in todo_args:
            res = create_masks(*args)
            results.append(res)
            if res["status"] == "ok":
                done += 1
            elif res["status"] == "error":
                errors += 1
            pbar.set_postfix(descargados=done, errores=errors, refresh=True)
            pbar.update(1)
        pbar.close()
        return skipped_results + results

    # Paralelo
    with Pool(processes=processes) as pool:
        pbar = tqdm(total=len(todo_args), disable=not show_progress, desc=desc, unit="par")
        for res in pool.imap_unordered(_worker, todo_args, chunksize=1):
            results.append(res)
            if res["status"] == "ok":
                done += 1
            elif res["status"] == "error":
                errors += 1
            pbar.set_postfix(descargados=done, errores=errors, last=res.get("msg", "")[:60], refresh=True)
            pbar.update(1)
        pbar.close()

    return skipped_results + results

In [ ]:
from pathlib import Path
from multiprocessing import cpu_count

DB_FOLDER = Path("../dataset_tfm/filtered_data")
S2_DIR = DB_FOLDER / "s2"
BUILD_DIR = DB_FOLDER / "building"; BUILD_DIR.mkdir(parents=True, exist_ok=True)
ROAD_DIR  = DB_FOLDER / "road";     ROAD_DIR.mkdir(parents=True, exist_ok=True)

s2_files = sorted(S2_DIR.glob("*.tif"))
pairs = []
for s2 in s2_files:
    token_parts = s2.stem.split("_")
    token = token_parts[1] if len(token_parts) > 1 else s2.stem
    out_build = BUILD_DIR / f"building_{token}.tif"
    out_road  = ROAD_DIR  / f"road_{token}.tif"
    pairs.append((str(s2), str(out_build), str(out_road)))

results = parallel_create_masks(
    pairs,
    processes=min(24, cpu_count()),
    pixel_size=2.5,
    out_shape=(256, 256),
    show_progress=True,
    desc="Procesando (descarga + rasterización)"
)

# Resumen
ok = sum(1 for r in results if r and r.get("status") == "ok")
sk = sum(1 for r in results if r and r.get("status") == "skipped")
er = sum(1 for r in results if r and r.get("status") == "error")
print(f"Resumen → Descargados/Procesados: {ok} | Ya existentes: {sk} | Errores: {er}")

Verificando existentes:   0%|          | 0/39877 [00:00<?, ?par/s]

Procesando (descarga + rasterización):   0%|          | 0/30089 [00:00<?, ?par/s]

In [1]:
# Get the mean and std for the s2 and s1 bands
import numpy as np
import rasterio as rio
def calculate_mean_std(files):
    means = []
    stds = []
    
    for file in files:
        with rio.open(file) as src:
            data = src.read()
            means.append(np.mean(data, axis=(1, 2)))
            stds.append(np.std(data, axis=(1, 2)))
    
    return np.mean(means, axis=0), np.mean(stds, axis=0)

import pathlib
S2_FOLDER = "../data/s2"
S1_FOLDER = "../data/s1"

s2_files = sorted(list(pathlib.Path(S2_FOLDER).glob("*.tif")))
s1_files = sorted(list(pathlib.Path(S1_FOLDER).glob("*.tif")))

s2_mean, s2_std = calculate_mean_std(s2_files)
s1_mean, s1_std = calculate_mean_std(s1_files)

print(f"[{str(s2_mean / 10_000).replace(' ', ',')}, {str(s1_mean).replace(' ', ',')}]")
print(f"[{str(s2_std / 10_000).replace(' ', ',')}, {str(s1_std).replace(' ', ',')}]")

[[0.12019355,0.10635965,0.08203992,0.23945379,0.2044379,,0.15833034], [,-9.21341266,-16.8054372,]]
[[0.06304411,0.05342577,0.04976318,0.081813,,,0.05458699,0.05351668], [4.07210856,4.13933744]]


In [ ]:
## Probar el ploteo de etiquetas